In [1]:
import zipfile
import os
import numpy as np
import pathlib
import pandas as pd
from math import ceil
import tensorflow as tf
import numpy as np
import IPython.display as display
import keras
from keras import backend as K
from matplotlib import pyplot as plt
from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10


print(tf.__version__)

root_path = './'

models_path = os.path.join(root_path, 'saved_models_new')
if not os.path.exists(models_path):
    os.mkdir(models_path)


tgt_pth = os.path.join(root_path, 'visualize_dcgan-v35')

if not os.path.exists(tgt_pth):
    os.mkdir(tgt_pth)



BATCH_SIZE = 128
img_rows, img_cols, img_depth = (32, 32, 3)
num_classes = 10


# The data, split between train and test sets:
(x_train, y_train), (_, _) = cifar10.load_data()


# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)


x_train = x_train.astype('float32')
x_train = x_train / 127.5 - 1

def rescaleFn(img):
    return img / 127.5 - 1


datagen=ImageDataGenerator()


# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

train_generator = datagen.flow(x_train, batch_size=BATCH_SIZE)

Using TensorFlow backend.


1.13.1


In [2]:
img_shape = (32, 32, 3)
img_len = np.prod(img_shape)
latent_dim = img_len
noise_len = 100
input_dim = noise_len
ngf = 32  # generator
ndf = 32  # discriminator
kernel_size = (5, 5)


def build_discriminator():
    return keras.Sequential([
        # conv block 1, (5, 5)@32
        keras.layers.Conv2D(
            filters=ndf,
            kernel_size=kernel_size,
            strides=2,
            input_shape=img_shape,
            use_bias=False,
            kernel_initializer=keras.initializers.glorot_normal(),
            padding='same'
        ),
        keras.layers.LeakyReLU(alpha=0.2),
        
        # conv block 2, (5, 5)@64
        keras.layers.Conv2D(
            filters=ndf * 2,
            kernel_size=kernel_size,
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.glorot_normal(),
            padding='same'
        ),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(alpha=0.2),
        
        # conv block 3, (5, 5)@128
        keras.layers.Conv2D(
            filters=ndf * 4,
            kernel_size=kernel_size,
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.glorot_normal(),
            padding='same'
        ),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(alpha=0.2),
        
        # conv block 4, (5, 5)@256
        keras.layers.Conv2D(
            filters=ndf * 8,
            kernel_size=kernel_size,
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.glorot_normal(),
            padding='same'
        ),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(alpha=0.2),
        
        # conv block 4
        keras.layers.Conv2D(
            filters=1,
            kernel_size=(1, 1),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.glorot_normal(),
            padding='same'
        ),
        
        #output
        keras.layers.Activation('sigmoid'),
        
        #flatten
        keras.layers.Flatten()
    ])


def build_generator():
    
    return keras.Sequential([
        # 100 noise to 1024
        keras.layers.Dense(2*2*256, input_shape=(input_dim,)),
        
        # reshape 1d to 3d
        keras.layers.Reshape((2, 2, 256)),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(alpha=0.2),
        
        # transpose conv block 1, (5, 5)@128
        keras.layers.Conv2DTranspose(
            filters=ngf * 4,
            kernel_size=kernel_size,
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.glorot_normal(),
            padding='same'
        ),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(alpha=0.2),
        
        # transpose conv block 2, (5, 5)@64
        keras.layers.Conv2DTranspose(
            filters=ngf * 2,
            kernel_size=(3, 3),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.glorot_normal(),
            padding='same'
        ),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(alpha=0.2),
        
        # transpose conv block 3, (5, 5)@32
        keras.layers.Conv2DTranspose(
            filters=ngf,
            kernel_size=(3, 3),
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.glorot_normal(),
            padding='same'
        ),
        keras.layers.BatchNormalization(),
        keras.layers.LeakyReLU(alpha=0.2),
        
        # transpose conv block 6
        keras.layers.Conv2DTranspose(
            filters=3,
            kernel_size=kernel_size,
            strides=2,
            use_bias=False,
            kernel_initializer=keras.initializers.glorot_normal(),
            padding='same'
        ),
        
        # output
        keras.layers.Activation('tanh')
    ])


discriminator = build_discriminator()
generator = build_generator()

generator.summary()
discriminator.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              103424    
_________________________________________________________________
reshape_1 (Reshape)          (None, 2, 2, 256)         0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 2, 2, 256)         1024      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 2, 2, 256)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 4, 4, 128)         819200    
_________________________________________________________________
batch_normalization_5 (Batch (None, 4, 4, 128)         512       
_________________________________________________________________
leak

In [3]:
GLR = 0.0002  # generator
DLR = 0.0002  # discriminator
LR = 0.00002


optim = keras.optimizers.Adam(LR, beta_1=0.5)


discriminator.compile(
    optimizer=optim,
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

# noise
z = keras.layers.Input(shape=(noise_len,))

# freeze discriminator
discriminator.trainable = False

# output
x = generator(z)
out = discriminator(x)

# GAN
gan = keras.models.Model(inputs=z, outputs=out)

gan.compile(
    optimizer=optim,
    loss=keras.losses.binary_crossentropy,
    metrics=['accuracy'])

In [4]:
def visualizeGAN(e, z_real, z_fake):

    fig, axes = plt.subplots(4, 5, figsize=(20, 18))

    r_real = 0
    r_fake = 0
    for row, axe in enumerate(axes):
        for col, cell in enumerate(axe):
            if row % 2 == 0:
                cell.imshow(
                    0.5 * z_real[r_real * 5 + col] + 0.5
                )
            else:
                cell.imshow(
                    0.5 * z_fake[r_fake * 5 + col] + 0.5
                )

            cell.axis("off")

        if row % 2 == 0:
            r_real += 1
        else:
            r_fake += 1

    plt.axis("off")
    plt.tight_layout()

    fig.savefig(os.path.join(tgt_pth, '{}.jpg'.format(str(e).zfill(3))))
    
    plt.close()

In [ ]:
EPOCHS = 250001
STEPS = 1


train_loss_g = []
train_loss_d = []

train_acc_g = []
train_acc_d = []


# to be visualized
vis_noise = np.random.normal(size=(10, noise_len,))

# fake and real label ( Label Smoothing & Label Flipping )
y_fake = np.zeros(shape=(BATCH_SIZE,))
y_real = np.ones(shape=(BATCH_SIZE,))

# epochs
for e in range(EPOCHS):

    #batches
    loss = []
    acc = []

    for p in range(STEPS * 4):
        
        z = np.random.normal(loc=0, scale=1, size=(BATCH_SIZE, noise_len))
        x_fake = generator.predict(z)
        x_real = next(train_generator)
        
        if x_real.shape[0] < BATCH_SIZE:
            x_real = next(train_generator)

        # add noise
#         x_real = x_real + np.random.normal(loc=0, scale=0.01, size=x_real.shape)
#         x_fake = x_fake + np.random.normal(loc=0, scale=0.01, size=x_fake.shape)
#         y_real = np.random.normal(loc=0.95, scale=0.05, size=(BATCH_SIZE,))
    
        # train
        loss_1, acc_1 = discriminator.train_on_batch(x_fake, y_fake)
        loss_2, acc_2 = discriminator.train_on_batch(x_real, y_real)
        

        batch_loss = 0.5 * (loss_1 + loss_2)
        batch_acc = 0.5 * (acc_1 + acc_2)

        loss.append(batch_loss)
        acc.append(batch_acc)

    train_loss_d.append(np.mean(np.array(loss)))
    train_acc_d.append(np.mean(np.array(acc)))

    #batches
    loss = []
    acc = []

    for p in range(STEPS):

        z = np.random.normal(loc=0, scale=1, size=(BATCH_SIZE, noise_len))
#         y_real = np.random.normal(loc=0.95, scale=0.05, size=(BATCH_SIZE,))
        
        # train
        loss_1, acc_1 = gan.train_on_batch(z, y_real)
        
        loss.append(loss_1)
        acc.append(acc_1)

    train_loss_g.append(np.mean(np.array(loss)))
    train_acc_g.append(np.mean(np.array(acc)))


    print("E: {}, [D ACC: %{:.2f}], [D LOSS: {:.2f}], [G ACC: %{:.2f}], [G LOSS: {:.2f}]".format(
          e,
          train_acc_d[-1] * 100,
          train_loss_d[-1] * 100,
          train_acc_g[-1] * 100,
          train_loss_g[-1] * 100
      ))

    if e % 200 == 0:
        ## visualize results
        viz_fake = generator.predict(vis_noise)
        x_real = next(train_generator)
        visualizeGAN(e, x_real, viz_fake)
        
## save model
pth = os.path.join(models_path, 'gan.h5')
gan.save(pth)

pth = os.path.join(models_path, 'generator-{}-{}-{}.h5'.format(e, train_loss_g[-1], train_acc_g[-1]))
generator.save(pth)

pth = os.path.join(models_path, 'discriminator.h5')
discriminator.save(pth)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


E: 0, [D ACC: %59.08], [D LOSS: 71.57], [G ACC: %8.59], [G LOSS: 140.94]
E: 1, [D ACC: %79.69], [D LOSS: 47.58], [G ACC: %5.47], [G LOSS: 178.60]
E: 2, [D ACC: %91.80], [D LOSS: 32.54], [G ACC: %0.78], [G LOSS: 198.27]
E: 3, [D ACC: %95.12], [D LOSS: 25.82], [G ACC: %0.78], [G LOSS: 224.96]
E: 4, [D ACC: %97.46], [D LOSS: 21.64], [G ACC: %0.78], [G LOSS: 246.02]
E: 5, [D ACC: %98.93], [D LOSS: 18.01], [G ACC: %0.00], [G LOSS: 243.33]
E: 6, [D ACC: %99.51], [D LOSS: 16.23], [G ACC: %0.00], [G LOSS: 259.14]
E: 7, [D ACC: %99.02], [D LOSS: 14.72], [G ACC: %0.78], [G LOSS: 260.94]
E: 8, [D ACC: %99.32], [D LOSS: 13.43], [G ACC: %0.00], [G LOSS: 278.44]
E: 9, [D ACC: %99.90], [D LOSS: 12.55], [G ACC: %0.78], [G LOSS: 292.50]
E: 10, [D ACC: %99.71], [D LOSS: 10.98], [G ACC: %0.00], [G LOSS: 308.09]
E: 11, [D ACC: %99.80], [D LOSS: 10.50], [G ACC: %0.00], [G LOSS: 317.13]
E: 12, [D ACC: %99.80], [D LOSS: 9.01], [G ACC: %0.00], [G LOSS: 328.51]
E: 13, [D ACC: %100.00], [D LOSS: 8.95], [G ACC: 

E: 112, [D ACC: %100.00], [D LOSS: 0.76], [G ACC: %0.00], [G LOSS: 543.67]
E: 113, [D ACC: %100.00], [D LOSS: 0.82], [G ACC: %0.00], [G LOSS: 521.25]
E: 114, [D ACC: %100.00], [D LOSS: 0.63], [G ACC: %0.00], [G LOSS: 524.40]
E: 115, [D ACC: %100.00], [D LOSS: 0.65], [G ACC: %0.00], [G LOSS: 539.56]
E: 116, [D ACC: %100.00], [D LOSS: 0.68], [G ACC: %0.00], [G LOSS: 539.54]
E: 117, [D ACC: %100.00], [D LOSS: 0.58], [G ACC: %0.00], [G LOSS: 539.14]
E: 118, [D ACC: %100.00], [D LOSS: 0.58], [G ACC: %0.00], [G LOSS: 536.14]
E: 119, [D ACC: %100.00], [D LOSS: 0.53], [G ACC: %0.00], [G LOSS: 558.53]
E: 120, [D ACC: %100.00], [D LOSS: 0.70], [G ACC: %0.00], [G LOSS: 531.67]
E: 121, [D ACC: %100.00], [D LOSS: 0.54], [G ACC: %0.00], [G LOSS: 568.58]
E: 122, [D ACC: %100.00], [D LOSS: 0.57], [G ACC: %0.00], [G LOSS: 541.78]
E: 123, [D ACC: %100.00], [D LOSS: 0.66], [G ACC: %0.00], [G LOSS: 545.01]
E: 124, [D ACC: %100.00], [D LOSS: 0.50], [G ACC: %0.00], [G LOSS: 556.89]
E: 125, [D ACC: %100.00],

E: 222, [D ACC: %100.00], [D LOSS: 0.17], [G ACC: %0.00], [G LOSS: 676.11]
E: 223, [D ACC: %100.00], [D LOSS: 0.15], [G ACC: %0.00], [G LOSS: 676.11]
E: 224, [D ACC: %100.00], [D LOSS: 0.13], [G ACC: %0.00], [G LOSS: 675.90]
E: 225, [D ACC: %100.00], [D LOSS: 0.17], [G ACC: %0.00], [G LOSS: 680.42]
E: 226, [D ACC: %100.00], [D LOSS: 0.13], [G ACC: %0.00], [G LOSS: 679.95]
E: 227, [D ACC: %100.00], [D LOSS: 0.20], [G ACC: %0.00], [G LOSS: 657.46]
E: 228, [D ACC: %100.00], [D LOSS: 0.16], [G ACC: %0.00], [G LOSS: 673.21]
E: 229, [D ACC: %100.00], [D LOSS: 0.16], [G ACC: %0.00], [G LOSS: 685.15]
E: 230, [D ACC: %100.00], [D LOSS: 0.13], [G ACC: %0.00], [G LOSS: 688.28]
E: 231, [D ACC: %100.00], [D LOSS: 0.14], [G ACC: %0.00], [G LOSS: 671.86]
E: 232, [D ACC: %100.00], [D LOSS: 0.14], [G ACC: %0.00], [G LOSS: 690.70]
E: 233, [D ACC: %100.00], [D LOSS: 0.15], [G ACC: %0.00], [G LOSS: 687.70]
E: 234, [D ACC: %100.00], [D LOSS: 0.20], [G ACC: %0.00], [G LOSS: 668.51]
E: 235, [D ACC: %100.00],

...

E: 8788, [D ACC: %100.00], [D LOSS: 0.23], [G ACC: %96.88], [G LOSS: 12.22]
E: 8789, [D ACC: %99.90], [D LOSS: 0.35], [G ACC: %94.53], [G LOSS: 13.32]
E: 8790, [D ACC: %99.80], [D LOSS: 0.40], [G ACC: %95.31], [G LOSS: 12.89]
E: 8791, [D ACC: %100.00], [D LOSS: 0.41], [G ACC: %92.19], [G LOSS: 26.53]
E: 8792, [D ACC: %100.00], [D LOSS: 0.10], [G ACC: %82.03], [G LOSS: 39.17]
E: 8793, [D ACC: %99.90], [D LOSS: 0.35], [G ACC: %96.09], [G LOSS: 12.40]
E: 8794, [D ACC: %100.00], [D LOSS: 0.14], [G ACC: %91.41], [G LOSS: 20.75]
E: 8795, [D ACC: %100.00], [D LOSS: 0.12], [G ACC: %91.41], [G LOSS: 25.92]
E: 8796, [D ACC: %100.00], [D LOSS: 0.34], [G ACC: %82.81], [G LOSS: 42.81]
E: 8797, [D ACC: %100.00], [D LOSS: 0.22], [G ACC: %95.31], [G LOSS: 14.69]
E: 8798, [D ACC: %99.90], [D LOSS: 0.27], [G ACC: %95.31], [G LOSS: 13.01]
E: 8799, [D ACC: %100.00], [D LOSS: 0.27], [G ACC: %94.53], [G LOSS: 19.35]
E: 8800, [D ACC: %100.00], [D LOSS: 0.19], [G ACC: %93.75], [G LOSS: 17.47]
E: 8801, [D ACC:

E: 8897, [D ACC: %100.00], [D LOSS: 0.05], [G ACC: %96.09], [G LOSS: 8.85]
E: 8898, [D ACC: %100.00], [D LOSS: 0.18], [G ACC: %94.53], [G LOSS: 11.57]
E: 8899, [D ACC: %100.00], [D LOSS: 0.07], [G ACC: %92.97], [G LOSS: 14.45]
E: 8900, [D ACC: %100.00], [D LOSS: 0.09], [G ACC: %95.31], [G LOSS: 11.16]
E: 8901, [D ACC: %100.00], [D LOSS: 0.10], [G ACC: %95.31], [G LOSS: 11.70]
E: 8902, [D ACC: %100.00], [D LOSS: 0.06], [G ACC: %92.97], [G LOSS: 14.27]
E: 8903, [D ACC: %100.00], [D LOSS: 0.13], [G ACC: %90.62], [G LOSS: 23.13]
E: 8904, [D ACC: %100.00], [D LOSS: 0.08], [G ACC: %98.44], [G LOSS: 8.51]
E: 8905, [D ACC: %100.00], [D LOSS: 0.27], [G ACC: %98.44], [G LOSS: 8.11]
E: 8906, [D ACC: %100.00], [D LOSS: 0.25], [G ACC: %83.59], [G LOSS: 48.11]
E: 8907, [D ACC: %99.90], [D LOSS: 0.11], [G ACC: %86.72], [G LOSS: 26.62]
E: 8908, [D ACC: %100.00], [D LOSS: 0.07], [G ACC: %90.62], [G LOSS: 19.55]
E: 8909, [D ACC: %100.00], [D LOSS: 0.06], [G ACC: %92.97], [G LOSS: 16.20]
E: 8910, [D ACC:

E: 9006, [D ACC: %100.00], [D LOSS: 0.21], [G ACC: %9.38], [G LOSS: 321.99]
E: 9007, [D ACC: %99.80], [D LOSS: 0.70], [G ACC: %87.50], [G LOSS: 39.00]
E: 9008, [D ACC: %99.90], [D LOSS: 0.51], [G ACC: %46.09], [G LOSS: 146.96]
E: 9009, [D ACC: %100.00], [D LOSS: 0.18], [G ACC: %67.19], [G LOSS: 67.91]
E: 9010, [D ACC: %99.80], [D LOSS: 0.72], [G ACC: %68.75], [G LOSS: 76.36]
E: 9011, [D ACC: %99.51], [D LOSS: 1.62], [G ACC: %0.00], [G LOSS: 794.03]
E: 9012, [D ACC: %100.00], [D LOSS: 0.10], [G ACC: %4.69], [G LOSS: 467.19]
E: 9013, [D ACC: %100.00], [D LOSS: 0.19], [G ACC: %22.66], [G LOSS: 267.95]
E: 9014, [D ACC: %100.00], [D LOSS: 0.31], [G ACC: %28.12], [G LOSS: 212.71]
E: 9015, [D ACC: %99.71], [D LOSS: 1.04], [G ACC: %24.22], [G LOSS: 223.10]
E: 9016, [D ACC: %100.00], [D LOSS: 0.46], [G ACC: %27.34], [G LOSS: 248.62]
E: 9017, [D ACC: %100.00], [D LOSS: 0.22], [G ACC: %31.25], [G LOSS: 197.49]
E: 9018, [D ACC: %95.90], [D LOSS: 11.07], [G ACC: %0.00], [G LOSS: 1544.54]
E: 9019, [

E: 9114, [D ACC: %100.00], [D LOSS: 0.12], [G ACC: %89.84], [G LOSS: 26.10]
E: 9115, [D ACC: %100.00], [D LOSS: 0.18], [G ACC: %54.69], [G LOSS: 102.67]
E: 9116, [D ACC: %100.00], [D LOSS: 0.16], [G ACC: %85.16], [G LOSS: 34.68]
E: 9117, [D ACC: %100.00], [D LOSS: 0.19], [G ACC: %95.31], [G LOSS: 17.91]
E: 9118, [D ACC: %100.00], [D LOSS: 0.25], [G ACC: %71.88], [G LOSS: 67.06]
E: 9119, [D ACC: %100.00], [D LOSS: 0.09], [G ACC: %76.56], [G LOSS: 57.94]
E: 9120, [D ACC: %99.90], [D LOSS: 0.40], [G ACC: %35.16], [G LOSS: 205.49]
E: 9121, [D ACC: %99.22], [D LOSS: 1.96], [G ACC: %93.75], [G LOSS: 14.69]
E: 9122, [D ACC: %100.00], [D LOSS: 0.15], [G ACC: %40.62], [G LOSS: 148.71]
E: 9123, [D ACC: %100.00], [D LOSS: 0.05], [G ACC: %69.53], [G LOSS: 72.72]
E: 9124, [D ACC: %100.00], [D LOSS: 0.03], [G ACC: %90.62], [G LOSS: 22.00]
E: 9125, [D ACC: %100.00], [D LOSS: 0.04], [G ACC: %89.84], [G LOSS: 25.90]
E: 9126, [D ACC: %100.00], [D LOSS: 0.09], [G ACC: %78.12], [G LOSS: 45.97]
E: 9127, [D

E: 9222, [D ACC: %100.00], [D LOSS: 0.19], [G ACC: %68.75], [G LOSS: 67.02]
E: 9223, [D ACC: %100.00], [D LOSS: 0.11], [G ACC: %78.12], [G LOSS: 43.83]
E: 9224, [D ACC: %100.00], [D LOSS: 0.05], [G ACC: %82.03], [G LOSS: 41.64]
E: 9225, [D ACC: %98.93], [D LOSS: 2.99], [G ACC: %0.00], [G LOSS: 1581.96]
E: 9226, [D ACC: %96.00], [D LOSS: 11.70], [G ACC: %1.56], [G LOSS: 904.70]
E: 9227, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %0.00], [G LOSS: 875.45]
E: 9228, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %13.28], [G LOSS: 567.39]
E: 9229, [D ACC: %100.00], [D LOSS: 0.00], [G ACC: %32.03], [G LOSS: 330.28]
E: 9230, [D ACC: %100.00], [D LOSS: 0.05], [G ACC: %47.66], [G LOSS: 241.48]
E: 9231, [D ACC: %100.00], [D LOSS: 0.10], [G ACC: %60.16], [G LOSS: 150.31]
E: 9232, [D ACC: %100.00], [D LOSS: 0.03], [G ACC: %64.84], [G LOSS: 100.66]
E: 9233, [D ACC: %100.00], [D LOSS: 0.05], [G ACC: %64.06], [G LOSS: 110.92]
E: 9234, [D ACC: %100.00], [D LOSS: 0.12], [G ACC: %52.34], [G LOSS: 156.12]
E: 92

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_loss_g, label="Generator Loss");
plt.plot(train_loss_d, label="Discriminator Loss");
plt.legend();

In [ ]:
plt.figure(figsize=(20, 18))
plt.plot(train_acc_g, label="Generator Accuracy");
plt.plot(train_acc_d, label="Discriminator Accuracy");
plt.legend();